In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/0814_4th Presentation/

/content/drive/MyDrive/Colab Notebooks/0814_4th Presentation


In [ ]:
pwd

'/content/drive/MyDrive/Colab Notebooks/0814_4th Presentation'

In [ ]:
pip install contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 32.7 MB/s eta 0:00:00


In [ ]:
pip install basemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 860.7/860.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx

In [ ]:
# Read shapefile and csv
Outer = gp.read_file("London shp/merged_london.shp")

In [ ]:
Road_within_Outer = gp.read_file("Road/oproad shp/Road_Outer/RoadLink_Outer_2.shp")

In [ ]:
CountPoint = pd.read_csv('Data/0-Count locations.csv')

In [ ]:
# Convert easting and northing to Points
geometry = [Point(xy) for xy in zip(CountPoint['Easting (UK Grid)'], CountPoint['Northing (UK Grid)'])]
CountPoint = gp.GeoDataFrame(CountPoint, geometry=geometry)

# Set the CRS for both GeoDataFrames to British National Grid (EPSG:27700)
CountPoint.crs = 'epsg:27700'
Outer.crs = 'epsg:27700'

In [ ]:
# Read the csv file
LondonAccident = pd.read_csv("Cycling/GOV_Selected_2/006_Cycle_All Accident_Weighted Accident Rate_category.csv")

# Randomly sample 100 rows
sampled_data = LondonAccident.sample(n=100, random_state=42)  # random_state is optional, but makes the random choice reproducible

# Extract the x and y values
x_sampled = sampled_data['location_easting_osgr'].values
y_sampled = sampled_data['location_northing_osgr'].values

<ipython-input-9-5e04f1f97c8f>:2: DtypeWarning: Columns (0,2,29) have mixed types. Specify dtype option on import or set low_memory=False.
  LondonAccident = pd.read_csv("Cycling/GOV_Selected_2/006_Cycle_All Accident_Weighted Accident Rate_category.csv")


In [ ]:
# 首先，为每一条路创建一个默认颜色映射
color_mapping = {road_id: 'grey' for road_id in Road_within_Outer['identifier']}

# 然后，为 LondonAccident 中的 road_identifier 更新颜色映射
for index, row in LondonAccident.iterrows():
    road_id = row['closest_road_identifier']
    rate = row['adjusted_weighted_accident_rate']

    if rate == 3:
        color = 'red'
    elif rate == 2:
        color = 'orange'
    else:
        color = 'grey'

    # 更新映射
    color_mapping[road_id] = color

# 使用映射为每一条路分配颜色
colors = [color_mapping[road_id] for road_id in Road_within_Outer['identifier']]

# 使用Geopandas绘制结果
# 创建图形对象
fig, ax = plt.subplots(figsize=(50, 50))

Road_within_Outer.plot(ax=ax, color=colors,linewidth=1, zorder=1)
Outer.plot(ax=ax, color='none', edgecolor='black',linewidth=3, label='Shapefile')

# Add the Google Map as a background using contextily
ctx.add_basemap(ax, crs=CountPoint.crs, source=ctx.providers.Stamen.TonerLite, alpha=1)


# 添加图例
ax.legend()
ax.grid(False)
ax.set_title("Cycle Accidents and Count Locations in London", fontsize=40)

# 隐藏坐标轴经纬度数字
ax.set_xticks([])
ax.set_yticks([])

# 保存为图片
##plt.savefig('Output_jpg/002_Cycle Accident_no basemap.png', bbox_inches='tight', dpi=600)

# 显示图形
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 首先，为每一条路创建一个默认颜色和线宽映射
color_mapping = {road_id: 'grey' for road_id in Road_within_Outer['identifier']}
linewidth_mapping = {road_id: 1 for road_id in Road_within_Outer['identifier']}

# 然后，为 LondonAccident 中的 road_identifier 更新颜色和线宽映射
for index, row in LondonAccident.iterrows():
    road_id = row['closest_road_identifier']
    rate = row['adjusted_weighted_accident_rate']

    if rate == 3:
        color = 'red'
        linewidth = 3
    elif rate == 2:
        color = 'orange'
        linewidth = 2
    else:
        color = 'grey'
        linewidth = 1

    # 更新映射
    color_mapping[road_id] = color
    linewidth_mapping[road_id] = linewidth

# 使用映射为每一条路分配颜色和线宽
colors = [color_mapping[road_id] for road_id in Road_within_Outer['identifier']]
linewidths = [linewidth_mapping[road_id] for road_id in Road_within_Outer['identifier']]

# 使用Geopandas绘制结果
# 创建图形对象
fig, ax = plt.subplots(figsize=(50, 50))

Road_within_Outer.plot(ax=ax, color=colors, linewidth=linewidths, zorder=1)
Outer.plot(ax=ax, color='none', edgecolor='black',linewidth=5, label='Shapefile')

# Add the Google Map as a background using contextily
ctx.add_basemap(ax, crs=CountPoint.crs, source=ctx.providers.Stamen.TonerLite, alpha=1)

# 添加图例
ax.legend()
ax.grid(False)
ax.set_title("Cycle Accidents and Count Locations in London", fontsize=40)

# 隐藏坐标轴经纬度数字
ax.set_xticks([])
ax.set_yticks([])

# 保存为图片
##plt.savefig('Output_jpg/002_Cycle Accident_no basemap.png', bbox_inches='tight', dpi=600)

# 显示图形
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 首先，为每一条路创建一个默认颜色和线宽映射
color_mapping = {road_id: 'blue' for road_id in Road_within_Outer['identifier']}
linewidth_mapping = {road_id: 1 for road_id in Road_within_Outer['identifier']}
alpha_mapping = {road_id: 1 for road_id in Road_within_Outer['identifier']}


# 然后，为 LondonAccident 中的 road_identifier 更新颜色和线宽映射
for index, row in LondonAccident.iterrows():
    road_id = row['closest_road_identifier']
    rate = row['adjusted_weighted_accident_rate']

    if rate == 3:
        color = 'red'
        linewidth = 3
        alpha = 1
    elif rate == 2:
        color = 'orange'
        linewidth = 2
        alpha = 1
    else:
        color = 'blue'
        linewidth = 0.1
        alpha = 0.1

    # 更新映射
    color_mapping[road_id] = color
    linewidth_mapping[road_id] = linewidth

# 使用映射为每一条路分配颜色和线宽
colors = [color_mapping[road_id] for road_id in Road_within_Outer['identifier']]
alphas = [alpha_mapping[road_id] for road_id in Road_within_Outer['identifier']]
linewidths = [linewidth_mapping[road_id] for road_id in Road_within_Outer['identifier']]

# 使用Geopandas绘制结果
# 创建图形对象
fig, ax = plt.subplots(figsize=(50, 50))

Road_within_Outer.plot(ax=ax, color=colors, linewidth=linewidths, alpha=alphas ,zorder=1)
Outer.plot(ax=ax, color='none', edgecolor='black',linewidth=3, label='Shapefile')

# Add the Google Map as a background using contextily
###ctx.add_basemap(ax, crs=CountPoint.crs, source=ctx.providers.Stamen.TonerLite, alpha=1)

# 添加图例
ax.legend()
ax.grid(False)
ax.set_title("Cycle Accidents and Count Locations in London", fontsize=40)

# 隐藏坐标轴经纬度数字
ax.set_xticks([])
ax.set_yticks([])

# 保存为图片
##plt.savefig('Output_jpg/002_Cycle Accident_no basemap.png', bbox_inches='tight', dpi=600)

# 显示图形
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 首先，为每一条路创建一个默认颜色和线宽映射
color_mapping = {road_id: 'green' for road_id in Road_within_Outer['identifier']}
linewidth_mapping = {road_id: 1 for road_id in Road_within_Outer['identifier']}
alpha_mapping = {road_id: 1 for road_id in Road_within_Outer['identifier']}

# 然后，为 LondonAccident 中的 road_identifier 更新颜色和线宽映射
for index, row in LondonAccident.iterrows():
    road_id = row['closest_road_identifier']
    rate = row['adjusted_weighted_accident_rate']

    if rate == 3:
        color = 'red'
        linewidth = 3
        alpha = 1
    elif rate == 2:
        color = 'orange'
        linewidth = 2
        alpha = 1
    else:
        color = 'green'
        linewidth = 1
        alpha = 1

    # 更新映射
    color_mapping[road_id] = color
    linewidth_mapping[road_id] = linewidth
    alpha_mapping[road_id] = alpha  # 添加这行代码以更新alpha_mapping

# 使用映射为每一条路分配颜色、线宽和透明度
colors = [color_mapping[road_id] for road_id in Road_within_Outer['identifier']]
alphas = [alpha_mapping[road_id] for road_id in Road_within_Outer['identifier']]
linewidths = [linewidth_mapping[road_id] for road_id in Road_within_Outer['identifier']]

# 使用Geopandas绘制结果
# 创建图形对象
fig, ax = plt.subplots(figsize=(50, 50))



ax.set_facecolor('black')
fig.patch.set_facecolor('black')


Road_within_Outer.plot(ax=ax, color=colors, linewidth=linewidths, alpha=alphas ,zorder=1)
Outer.plot(ax=ax, color='none', edgecolor='green',linewidth=3, label='Shapefile')

##ctx.add_basemap(ax, crs=CountPoint.crs, source=ctx.providers.Stamen.TonerLite, alpha=0.1)
ctx.add_basemap(ax, crs=CountPoint.crs, source=ctx.providers.Stamen.Terrain, alpha=1)


# 添加图例
ax.legend()
ax.grid(False)
ax.set_title("Cycle Accidents and Count Locations in London", fontsize=40)

# 隐藏坐标轴经纬度数字
ax.set_xticks([])
ax.set_yticks([])

###plt.savefig('Output_jpg/003_Cycle Accident_no basemap.png', bbox_inches='tight', dpi=150)

# 显示图形
plt.show()


Output hidden; open in https://colab.research.google.com to view.

Output hidden; open in https://colab.research.google.com to view.